In [ ]:
import uproot
import json
import cabinetry
import hist

In [ ]:
def extract_samples_from_json(json_file):
    output_files = []
    with open(json_file, 'r') as fd:
        data = json.load(fd)
        for sample, conditions in data.items():
            for condition in conditions:
                sample_name = f"everything_merged_{sample}__{condition}.root"
                output_files.append(sample_name)
    return output_files
json_file = "nanoaod_inputs.json"
# LIST_OF_FILES_PER_SAMPLE is a list of 9 files containing histograms per sample
LIST_OF_FILES_PER_SAMPLE = extract_samples_from_json(json_file)
print(LIST_OF_FILES_PER_SAMPLE)
with uproot.recreate("histograms.root") as f_out:
    for h_file in LIST_OF_FILES_PER_SAMPLE:
        with uproot.open(h_file) as f_per_sample:
            for key in f_per_sample.keys(cycle=False):
                value = f_per_sample[key]
                f_out[key] = value

In [ ]:
import utils.rebinning  # noqa: E402

cabinetry_config = cabinetry.configuration.load("cabinetry_config.yml")

# rebinning: lower edge 110 GeV, merge bins 2->1
rebinning_router = utils.rebinning.get_cabinetry_rebinning_router(cabinetry_config, rebinning=slice(110j, None, hist.rebin(2)))
cabinetry.templates.build(cabinetry_config, router=rebinning_router)
cabinetry.templates.postprocess(cabinetry_config)  # optional post-processing (e.g. smoothing)
ws = cabinetry.workspace.build(cabinetry_config)
cabinetry.workspace.save(ws, "workspace.json")

In [ ]:
!pyhf inspect workspace.json | head -n 20
